## Méthodes d'optimisation stochastique

### Huc-Lhuillery Alexia, groupe B2

## I. Minimisation stochastique d'une fonction déterministe


On s'intéresse au problème $\min f(x) =\max_{i=1\dots m}|a_i^Tx-b_i | = \max_{i=1\dots m} |(Ax-b)_i|.$ 

On suppose que $a_i$ est un vecteur colonne représentant la $i$eme ligne d'une matrice $A$ de taille $m \times n$ ($m=100, n=20$) , et les $b_i$ sont les composantes d'un second membre $b$ de taille $m$, ($1 \le i \le m$), $x$ un vecteur de taille $n$. Ces quantités sont générées une fois pour toutes à partir de distributions Gaussiennes de moyenne nulle et d'écart type identité.

**Question 1 :** Constuire $A$ et $b$. Proposer le calcul d'un sous-gradient en $x$ de $f$. On pourra utiliser la fonction findmax de Julia. 


**Réponse :** Calcul du sous-gradient :

Pour la valeur absolue on peut prendre comme sous-gradient le signe de la valeur, et pour le maximum, la valeur du résultat du maximum étant l'un des coefficient $y_i = |(Ax-b)_i|$, le sous-gradient vaut donc $a_i$ la $i$-ème ligne de la matrice $A$, qui est la dérivée de l'expression $(Ax-b)_i$ en $x$. 

On obtient alors comme valeur d'un sous-gradient de $f$ en $x$ : 

$$ signe(f(x)) a_i \mbox{  avec  } i \mbox{  tel que  } f(x) = \mid (Ax-b)_i \mid $$

In [8]:
import Pkg
Pkg.add("Distributions")
Pkg.add("JuMP")
Pkg.add("GLPK")
Pkg.add("Plots")
Pkg.add("PlotlyBase")
using PlotlyBase
using JuMP
using GLPK
using Plots
using Distributions
using LinearAlgebra

   Updating registry at `~/.julia/registries/General`
┌ Warning: Some registries failed to update:
│     — `~/.julia/registries/General` — registry dirty
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.4/Pkg/src/Types.jl:1131
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
  [31c24e10] + Distributions v0.25.45
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [b429d917] + DensityInterface v0.4.0
  [31c24e10] + Distributions v0.25.45
  [1a297f60] + FillArrays v0.12.8
  [90014a1f] + PDMats v0.11.16
  [1fd47b50] + QuadGK v2.6.0
  [79098fc4] + Rmath v0.6.1
  [f50d1b31] + Rmath_jll v0.2.2+2
  [276daf66] ↓ SpecialFunctions v2.1.7 ⇒ v1.8.7
  [4c63d2b9] + StatsFuns v0.9.7
  [4607b0f0] + SuiteSparse 
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.jul

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [9]:
# Construction des données A  et b
n = 20;
m = 100;

A = randn(m,n)
b = randn(m,1)
x = zeros(n,1)

# Fonction calculant la valeur de f en x 
function fval(A,b,x)
    return maximum(abs.(A*x - b))
end

# Fonction calculant un sous-gradient en x de f
function subgrad(A,b,x) 
    fx = A*x - b
    (~,ind) = findmax(fx)
    i = ind[1]
    return sign(fx[i]) .* (A[i,:])
end

subgrad (generic function with 1 method)

**Question 2 :** Ce problème peut se reformuler comme un problème de programmation linéaire : $$(\mathcal{P}_{lp})\quad \left\{ \begin{array}{c} \min_{(x,R)\in \mathbb{R}^n\times \mathbb{R}} h(x,R)=R\\
s.c. \quad-R*e\leq A*x-b\leq R*e\end{array}\right.$$ avec $e=[1,\cdots,1]^T\in \mathbb{R}^m$. Résoudre le problème $(\mathcal{P}_{lp})$ en utilisant le solveur "GLPK" de la librairie JuMP. Plus d'informations sont disponibles ici :  http://www.juliaopt.org/JuMP.jl/latest/quickstart/

In [10]:
# Définition du modèle
model = Model(GLPK.Optimizer);

# Définition des variables d'optimisation
@variable(model, x[1:n]);
@variable(model, R);

# Définition de la fonctionnelle à minimiser
@objective(model, Min, R);

# Définition des contraintes
@constraint(model, c1[i=1:m], -R <= sum(A[i,j]*x[j] for j in 1:n) - b[i]);
@constraint(model, c2[i=1:m], R >= sum(A[i,j]*x[j] for j in 1:n) - b[i]);
        
# Résolution
optimize!(model);
        
# Résultats à optimalité 
println(solution_summary(model))
xstar = value.(x)
Rstar = value(R)

println("The function value at the solution is: ",Rstar, " or ",findmax(abs.(A*xstar-b)))


* Solver : GLPK

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "Solution is optimal"

* Candidate solution
  Objective value      : 1.5149792153189994
  Objective bound      : -Inf
  Dual objective value : 1.514979215318999

* Work counters
  Solve time (sec)   : 0.01294

The function value at the solution is: 1.5149792153189994 or (1.514979215319, CartesianIndex(47, 1))


**Question 3 :** Résoudre le problème en utilisant un algorithme de sous-gradient. Dans un premier temps vous utiliserez un sous-gradient exact (Question 1), puis vous introduirez un bruit artificiel qui suit une distribution normale de moyenne nulle et d'écart-type $3 e-1$.

**Question 4 :** Vous afficherez les courbes de convergence de $f_{best}^k-f_{star}$, avec $f_{star}$ obtenue à la Question 1. 

In [11]:
#Initialisation
x = zeros(n,1);
fbest =1e10; # $f_{best}^0$: cas du sous-gradient exact
fbestp=1e10; # $f_{best}^0$: cas du sous-gradient bruité
histo =[];# Suite des itérés f_{best}^k-f_{star} pour le cas du sous-gradient exact
histop=[];# Suite des itérés f_{best}^k-f_{star}, pour le cas du sous-gradient bruité

#Niveau de bruit
noise_lvl=.3;

# Resolution 
# Initialisation des variables 
itermax = 100;
xk = x;
xpk = x;
# Rstar est le résultat obtenu par le solveur GLPK 
f_star = Rstar;

for i in 1:itermax
    alpha = 1/(100*i);

    # Calcul de l'itération avec le gradient exact 
    
    feval = fval(A,b,xk);
    if feval < fbest
        # Mise-à-jour de f_{best}
        fbest = feval;
    end
    g = subgrad(A,b,xk);
    xk = xk - alpha*g;
    # Stockage
    append!(histo, fbest-f_star)
    
    # Calcul de l'itération avec le gradient bruité 
    
    fevalp = fval(A,b,xpk);
    if fevalp < fbestp
        # Mise-à-jour de fp_{best}
        fbestp = fevalp;
    end
    
    # Calcule un sous-gradient et y ajoute un bruit 
    # de niveau de bruit donné noise_lvl
    gp = subgrad(A,b,xpk) + noise_lvl*randn(n,1);
    xpk = xpk - alpha*gp;
    # Stockage
    append!(histop, fbestp-f_star)
    
end

#Affichage des courbes de convergence
plotly();
iter=1:itermax;
hf=[histo,histop];
plot(iter,hf,title="Convergence curves",label=["Exact" "Noisy"],lw=3)

<!DOCTYPE html>
 
 
 Plots.jl

**Observations :**

L'ajout de bruit induit une différence au bout d'une vingtaine d'itérations entre les deux courbes, qui reste ensuite relativement constante. Les deux courbes se rapprochent tout de mêmes de 0, ce qui indique que les deux méthodes permettent de se rapprocher de la solution $f_{star}$ du problème, la méthode utilisant un sous-gradient exact donnant un résultat plus proche de la solution réelle $f_{star}$ que celle utilisant un sous-gradient bruité. 

## II. Minimisation stochastique d'une fonction stochatique



On s'intéresse au problème
$$\min_x f(x) =\text{E} (\max_{i=1\dots m}|a_i^Tx-b_i |).$$

On suppose que $a_i$ est un vecteur colonne représentant la $i$eme ligne d'une matrice $A$ de taille $m \times n$ ($m=100, n=20$) , et les $b_i$ sont les composantes d'un second membre $b$ de taille $m$, ($1 \le i \le m$), $x$ un vecteur de taille $n$. Ces quantités sont générées une fois pour toutes à partir de distributions Gaussiennes de moyenne connue $\bar{A}$ et $\bar{b}$ (non nécesairement nulle) et d'écart type identité.

**Question 5 :** Proposer deux fonctions d'évaluation de la fonction $f$ et d'un sous-gradient de $f$ basées sur des échantillons de taille $M$.


In [12]:
# Evaluation de f
function fvals(Abar,bbar,noise,xs,M) 
    # Abar, bbar : moyenne des données
    # noise : niveau de bruit 
    # xs : vecteur courant 
    # M: taille de l'échantillon

    somme = 0;
    
    # On fait une moyenne sur M observations 
    # de l'évaluation de f en x, avec les données Abar
    # et bbar bruitées d'un bruit noise
    
    for k in 1:M 
        somme += fval(Abar + noise*randn(m,n), bbar + noise*randn(m), xs);
    end
    res = somme/M;
    return res
    
end

# Evaluation d'un sous-gradient
function subgrads(Abar,bbar,noise,xs,M)
    # Abar, bbar : moyenne des données
    # noise : niveau de bruit 
    # xs : vecteur courant 
    # M: taille de l'échantillon
    
    somme = zeros(n,1);
    
    # On fait une moyenne sur M observations
    # de l'évaluation de gradf en x, avec les données Abar
    # et bbar bruitées d'un bruit noise
    
    for k in 1:M
        somme += subgrad(Abar + noise*randn(m,n), bbar + noise*randn(m), xs);
    end
    res = somme/M;
    return res

end

subgrads (generic function with 1 method)

**Question 6 :** Comparer les courbes de convergence du problème déterministe $$ \min_x f(x) = \max_{i=1\dots m}|\text{E} (a_i)^Tx-\text{E} (b_i) |,$$ et du problème stochastique obtenu avec $M=10,100,1000$ échantillons.

In [13]:
# Données
Abar=10*ones(m,n)+1*randn(m,n);
bbar=10*randn(m,1);

#x_0
xd = ones(n,1); # problème déterministe
xs = xd;        # problème stochastique

# Bruit et echantillon
M  = 100;
noise  = 4;

fbestd =1e10; # $f_{best}^0$: cas d'une résolution déterministe
fbests =1e10; # $f_{best}^0$: cas d'une résolution stochastique
histod =[]; # Suite des itérés f_{best}^k pour le cas d'une résolution déterministe
histos =[]; # Suite des itérés f_{best}^k pour le cas d'une résolution stochastique

# Résolution 
# Initialisation des variables 
itermax = 1000;
xdk = xd;
xsk = xs;

for i in 1:itermax
    alpha = 1/(100*i);

    # Calcul de l'itération avec 
    # le gradient deterministe 

    fevald = fval(Abar,bbar,xdk);
    if fevald < fbestd
        # Mise-à-jour de fd_{best}
        fbestd = fevald;
    end
    gd = subgrad(Abar,bbar,xdk);
    xdk = xdk - alpha*gd;
    # Stockage
    append!(histod, fbestd)

    # Calcul de l'itération avec 
    # le gradient stochastique

    fevals = fvals(Abar,bbar,noise,xsk,M);
    if fevals < fbests
        # Mise-à-jour de fs_{best}
        fbests = fevals;
    end
    gs = subgrads(Abar,bbar,noise,xsk,M);
    xsk = xsk - alpha*gs;
    # Stockage
    append!(histos, fbests)
end
# Fin insérer code
#Affichage
#plotly();
iter=1:itermax;
hf=[histod,histos];
plot(iter,hf,title="Convergence curves",label=["Deterministic" "Stochastic"],lw=3)

<!DOCTYPE html>
 
 
 Plots.jl

**Observations :**

La courbe obtenue avec la méthode stochastique est de plus en plus lisse avec une augmentation de la valeur de M. Le résultat est différents du résultat obtenu avec la méthode déterministe car les données sont bruitées, et leur écart devient plus important au fil des itérations avec toutes les valeurs de M. 

**Question 7 :** Répéter les expériences et comparer les valeurs meilleurs valeurs de f obtenues ($f_{best}$) aprs un nombre fixé d'itérations. 

In [14]:
# Données
Abar=10*ones(m,n)+1*randn(m,n);
bbar=10*randn(m,1);

# x_0
xd = ones(n,1); # résolution déterministe
xs = xd;        # résolution stochastique

# Bruit et echantillon
M  = 200;
noise  = 4;

#Nombre d'itérations
niter=100;

fbestd =1e10; # $f_{best}^0$: cas d'une résolution déterministe
fbests =1e10; # $f_{best}^0$: cas d'une résolution stochastique
fbesttd=[]; # f_{best} pour chaque expériences dans le cas d'une résolution déterministe
fbestts=[]; # f_{best} pour chaque expériences dans le cas d'une résolution stochastique

for nexp=1:20
    # Répétition des expériences    
    fbestd =1e10; # $f_{best}^0$: cas d'une résolution déterministe
    fbests =1e10; # $f_{best}^0$: cas d'une résolution stochastique
    xdk = xd; 
    xsk = xs;
    for i in 1:niter
        alpha = 1/i;

        # Calcul de l'itération avec 
        # le gradient deterministe 
    
        fevald = fval(Abar,bbar,xdk);
        if fevald < fbestd
            # Mise-à-jour de fd_{best}
            fbestd = fevald;
        end
        gd = subgrad(Abar,bbar,xdk);
        xdk = xdk - alpha*gd;
    
        # Calcul de l'itération avec 
        # le gradient stochastique 
    
        fevals = fvals(Abar,bbar,noise,xsk,M);
        if fevals < fbests
            # Mise-à-jour de fs_{best}
            fbests = fevals;
        end
        gs = subgrads(Abar,bbar,noise,xsk,M);
        xsk = xsk - alpha*gs;

    end
    # Stockage
    append!(fbesttd, fbestd)
    append!(fbestts, fbests)

end

#Affichage
#plotly();
iter=1:20;
hf=[fbesttd,fbestts];
plot(iter,hf,title="Convergence curves",label=["Deterministic" "Stochastic"],lw=3)

<!DOCTYPE html>
 
 
 Plots.jl

**Observations :**

La méthode stochastique donne des résultats très différents pour plusieurs expériences avec les mêmes paramètres, qui se rapprochent plus ou moins de la solution obtenue par méthode déterministe. En moyenne, le résultat obtenu par la méthode stochastique est plus élevé que le résultat obtenu par la méthode déterministe. 